# House Extractor

In [26]:
from bs4 import BeautifulSoup
import requests
import csv
import os
import pandas as pd
import shutil

In [17]:
# to be clean creating a python "switch" statement:

def switch_unlabeledFields(args):
    switcher = {1: 'Address:',
                2: 'Room:',
                3: 'City:',
                4: 'Province:',
                5: 'Post:',
                9: 'Street:',
                10: 'District:',
                14: 'Year:'
                16: 'house_type:',
                17: 'house_style:'}
    return switcher.get(args, str(args) + ':')

In [40]:
house_detail_dictionary = {}


with open('house_data.csv', 'a') as csv_file:    
    for filename in os.listdir(os.getcwd()):
        
        if filename.endswith('.html'):
            with open(filename) as html_file:
                
                soup = BeautifulSoup(html_file, 'lxml')
                date_printed = soup.find('div', class_='header').text
                date_printed = (date_printed[date_printed.find('Printed') + 11:date_printed.find('PM')-10])
                source = filename
                div_house_list = [tag for tag in soup.find_all('div', class_='report-container') if len(tag.get('class')) > 1]
                #report-container has multiple classes so have to specify only pick those with multiple classes
                for div_house in div_house_list:
                    house_spans = div_house.find_all('span', class_='formitem formfield')
                    address = div_house.find('div')
                    i = 0
                    for span in house_spans:
                        i = i + 1
                        label = span.label
                        detail = span.find('span', class_='value')
                        if label is not None:
                            house_detail_dictionary[label.text[:-1]] = detail.text
                        if label is None and i <= 30:
                            label = switch_unlabeledFields(i)                
                            house_detail_dictionary[label[:-1]] = detail.text
                    house_detail_dictionary['DateAdded'] = date_printed
                    house_detail_dictionary['Source'] = source
                    w = csv.DictWriter(csv_file, house_detail_dictionary.keys(), lineterminator='\n')
                    w.writerow(house_detail_dictionary)
            shutil.move(os.getcwd() + "\\" + filename, os.getcwd() + "\Archive\\" + filename)